In [ ]:
!pip install streamlit transformers datasets scikit-learn
!pip install --no-cache-dir accelerate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import string

# Load the datasets
training_data_path = '/content/drive/MyDrive/training_data.csv'
training_data_pd = pd.read_csv(training_data_path)

# Function to clean text
def clean_text(text):
    text = text.lower()  # Convert text to lowercase
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = " ".join(text.split())  # Remove extra whitespace
    return text

# Apply cleaning function to the sentence column
training_data_pd['sentence'] = training_data_pd['sentence'].apply(clean_text)

# Display the first few rows of the cleaned data to confirm changes
print(training_data_pd.head())


   id                                           sentence difficulty
0   0  les coûts kilométriques réels peuvent diverger...         C1
1   1  le bleu cest ma couleur préférée mais je naime...         A1
2   2  le test de niveau en français est sur le site ...         A1
3   3             estce que ton mari est aussi de boston         A1
4   4  dans les écoles de commerce dans les couloirs ...         B1


In [ ]:
from transformers import CamembertTokenizer, CamembertForSequenceClassification
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Load the tokenizer
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

# Function to encode data
def encode_data(tokenizer, df):
    texts = df['sentence'].tolist()
    labels = df['difficulty'].map({'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}).tolist()
    encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=128)
    return Dataset.from_dict({
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': labels
    })

# Split the DataFrame into training and validation sets
train_data, val_data = train_test_split(training_data_pd, test_size=0.10, random_state=42)

# Tokenize and prepare datasets
train_dataset = encode_data(tokenizer, train_data)
val_dataset = encode_data(tokenizer, val_data)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
from datasets import load_metric
import numpy as np

# Load the model
model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=6)

# Metric for evaluation
def compute_metrics(eval_pred):
    metric = load_metric("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = metric.compute(predictions=predictions, references=labels)
    return {"accuracy": accuracy['accuracy']}

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,  # Reduced epochs
    per_device_train_batch_size=16,  # Adjust batch size
    warmup_steps=500,  # Fewer warmup steps
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    learning_rate=5e-5,  # Adjust learning rate
    fp16=False,  # Disable FP16 since it's not supported on CPUs
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    greater_is_better=True
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Early stopping to avoid overfitting
)

# Train the model
trainer.train()

# Save the model and tokenizer
model.save_pretrained("./model")
tokenizer.save_pretrained("./model")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
1,1.368200,1.322310,0.460417
2,1.102700,1.077510,0.514583
3,0.974400,1.055678,0.531250


<ipython-input-5-1237399319cb>:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric 

('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/sentencepiece.bpe.model',
 './model/added_tokens.json')

In [ ]:
%%writefile app.py
import streamlit as st
from transformers import CamembertTokenizer, CamembertForSequenceClassification
import torch
import numpy as np

# Load the tokenizer and model
model_path = "./model"
tokenizer = CamembertTokenizer.from_pretrained(model_path)
model = CamembertForSequenceClassification.from_pretrained(model_path)

# Define a function to predict the difficulty
def predict_difficulty(text):
    # Tokenize the input text
    inputs = tokenizer(text, truncation=True, padding='max_length', max_length=128, return_tensors="pt")

    # Ensure the model is in evaluation mode and make predictions
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=-1).item()
    levels = ['A1', 'A2', 'B1', 'B2', 'C1', 'C2']
    return levels[predicted_class_id]

# Streamlit app interface
st.title("French Text Difficulty Predictor")
st.write("Enter a French text and get its difficulty level predicted.")

# Text input
user_input = st.text_area("Enter French text here:", "")

# Predict button
if st.button("Predict Difficulty"):
    if user_input.strip() == "":
        st.write("Please enter a valid French text.")
    else:
        # Predict difficulty
        difficulty = predict_difficulty(user_input)
        st.write(f"The predicted difficulty level is: **{difficulty}**")


Writing app.py


In [ ]:
!jupyter nbconvert --to script Streamlit_App.ipynb


[NbConvertApp] WARNING | pattern 'Streamlit_App.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
 

In [ ]:
!mv Streamlit_TEST.py app.py


mv: cannot stat 'Streamlit_TEST.py': No such file or directory
